<a href="https://colab.research.google.com/github/PatoJosefo/API-NEXUS/blob/main/colab_unificado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# CÉLULA 1: Importações
# ===============================
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output,Javascript
from google.colab import drive
import seaborn as sns # Importar seaborn para facilitar o agrupamento
import warnings

# Opcional: Ignorar avisos específicos se forem muito frequentes, por exemplo, FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print("Célula 1/11: Bibliotecas importadas.")

In [ ]:
# ===============================
# CÉLULA 2: Montar Google Drive & Definir Caminho
# ===============================
# Montar Google Drive para acessar seus arquivos
# Pode ser necessário autorizar o acesso do Colab na primeira vez
try:
    drive.mount('/content/drive', force_remount=True) # force_remount pode ser útil
    print("Google Drive montado com sucesso.")
except Exception as e:
    print(f"Erro ao montar Google Drive: {e}")
    print("Por favor, verifique se você autorizou o acesso e se o caminho do Drive está correto.")

# !!! IMPORTANTE !!!
# ATUALIZE ESTE CAMINHO se sua pasta 'API1S' estiver localizada em outro lugar no seu Google Drive
DRIVE_BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/API1S/"
print(f"Usando caminho base para os dados: {DRIVE_BASE_PATH}")
print("--> Verifique se este caminho aponta para sua pasta contendo os arquivos CSV <--")

print("\nCélula 2/11: Tentativa de montagem do Drive e definição do caminho base concluída.")

In [ ]:
# ===============================
# CÉLULA 3: Carregar Dados Auxiliares
# ===============================
# Carregar arquivos CSV auxiliares (Municípios e códigos NCM/SH)

print("Carregando dados auxiliares...")
# Definir caminhos dos arquivos usando o caminho base
municipios_file = f"{DRIVE_BASE_PATH}UF_MUN.csv"
ncm_sh_file = f"{DRIVE_BASE_PATH}NCM_SH.csv"

try:
    print(f"Tentando carregar: {municipios_file}")
    municipios_df = pd.read_csv(
        municipios_file,
        delimiter=';',
        encoding='latin-1'
    )
    # Limpar nomes das colunas imediatamente após o carregamento
    municipios_df.columns = municipios_df.columns.str.strip()
    print(f"Carregado UF_MUN.csv ({len(municipios_df)} rows)")

    print(f"Tentando carregar: {ncm_sh_file}")
    ncm_sh_df = pd.read_csv(
        ncm_sh_file,
        delimiter=';',
        encoding='latin-1'
    )
    # Limpar nomes das colunas imediatamente após o carregamento
    ncm_sh_df.columns = ncm_sh_df.columns.str.strip()
    print(f"Carregado NCM_SH.csv ({len(ncm_sh_df)} rows)")

    # --- Limpeza de Dados para os DataFrames Auxiliares ---
    print("Limpando dados auxiliares...")
    # Garantir que chaves de junção sejam de tipos compatíveis e tratar erros
    municipios_df['CO_MUN_GEO'] = pd.to_numeric(municipios_df['CO_MUN_GEO'], errors='coerce')
    rows_before = len(municipios_df)
    municipios_df.dropna(subset=['CO_MUN_GEO'], inplace=True) # Remover linhas onde CO_MUN_GEO não é numérico
    print(f"  Municípios: Removidas {rows_before - len(municipios_df)} linhas com CO_MUN_GEO inválido.")
    municipios_df['CO_MUN_GEO'] = municipios_df['CO_MUN_GEO'].astype(int)

    ncm_sh_df['CO_SH4'] = pd.to_numeric(ncm_sh_df['CO_SH4'], errors='coerce')
    rows_before = len(ncm_sh_df)
    ncm_sh_df.dropna(subset=['CO_SH4'], inplace=True) # Remover linhas onde CO_SH4 não é numérico
    print(f"  NCM_SH: Removidas {rows_before - len(ncm_sh_df)} linhas com CO_SH4 inválido.")
    ncm_sh_df['CO_SH4'] = ncm_sh_df['CO_SH4'].astype(int)

    print("Dados auxiliares carregados e limpos com sucesso.")
    # Exibir informações para verificação (opcional, mas recomendado)
    # print("\n_Municipios DataFrame Info:_")
    # municipios_df.info()
    # print("\n_NCM SH DataFrame Info:_")
    # ncm_sh_df.info()

except FileNotFoundError as e:
    print(f"\n--- ERRO ---")
    print(f"Erro ao carregar arquivo auxiliar: {e}")
    print(f"Por favor, verifique se '{e.filename}' existe no caminho especificado: {DRIVE_BASE_PATH}")
    print(f"Verifique o DRIVE_BASE_PATH na Célula 2 e os nomes dos arquivos.")
    print(f"-------------\n")
    # Gerar um erro ou definir dataframes vazios para evitar falhas posteriores
    municipios_df = pd.DataFrame()
    ncm_sh_df = pd.DataFrame()
except Exception as e:
    print(f"\n--- ERRO ---")
    print(f"Ocorreu um erro inesperado ao carregar dados auxiliares: {e}")
    print(f"-------------\n")
    municipios_df = pd.DataFrame()
    ncm_sh_df = pd.DataFrame()


print("\nCélula 3/11: Tentativa de carregamento de dados auxiliares concluída")

In [ ]:
# ===============================
# CÉLULA 4: Definição da Função de Carregamento de Dados
# ===============================
# Função para carregar e pré-processar dados anuais de exportação

def carregar_dados_anuais(anos_lista):
    """
    Carrega dados de exportação para os anos especificados, filtra para SP, limpa e pré-processa

    Argumentos:
        anos_lista (lista): Uma lista de strings representando os anos para carregar (exemplo: ['2019', '2020']).

    Devoluções:
        dicionário: A dictionary where keys are years (str) and values are preprocessed pandas DataFrames for SP.
              Retorna None se os dados auxiliares não tiverem sido carregados corretamente.
    """
    dados = {}
    print(f"Iniciando carregamento de dados para os anos: {anos_lista}...")

    # --- Pré-verificação: garantir que os DataFrames auxiliares existem ---
    # Usar 'globals()' para verificar se as variáveis existem no escopo global
    if 'municipios_df' not in globals() or municipios_df.empty or \
       'ncm_sh_df' not in globals() or ncm_sh_df.empty:
         print("\n--- ERRO ---")
         print("Dados auxiliares (municipios_df ou ncm_sh_df) não foram carregados corretamente ou estão vazios.")
         print("Certifique-se de que a Célula 3 foi executada sem erros antes de rodar esta célula")
         print("-------------\n")
         return None # Indicar falha crítica


    for ano in anos_lista:
        file_path = f"{DRIVE_BASE_PATH}EXP_{ano}_MUN.csv" # Usar o caminho base definido na Célula 2
        print(f"Tentando carregar: {file_path}")
        try:
            # Carregar com dtypes especificados, depois limpar colunas
            df = pd.read_csv(
                file_path,
                delimiter=';',
                dtype={'KG_LIQUIDO': str, 'VL_FOB': str, 'CO_MUN': str, 'SH4': str}, # Ler colunas chave como string inicialmente
                encoding='latin-1' # Manter codificação consistente
            )
            df.columns = df.columns.str.strip() # Limpar nomes das colunas
            print(f"Carregado {ano}, processando...")

            # --- Filtrar para Municípios de SP ---
            df_sp = pd.DataFrame() # Inicializar DataFrame vazio
            if 'SG_UF_MUN' in df.columns:
                 # Método principal: Usar SG_UF_MUN se disponível
                 df_sp = df[df['SG_UF_MUN'].str.contains('SP', case=False, na=False)].copy() # case=False para robustez
                 print(f"  Filtrado para SP usando 'SG_UF_MUN': {len(df_sp)} linhas encontradas.")
            elif 'CO_MUN' in df.columns and 'municipios_df' in globals() and not municipios_df.empty:
                 # Método alternativo: Usar CO_MUN e o dataframe auxiliar municipios_df
                 print("'SG_UF_MUN' não encontrado ou inutilizável. Filtrando SP baseado em 'CO_MUN' usando dados auxiliares.")
                 sp_codes = municipios_df[municipios_df['SG_UF'] == 'SP']['CO_MUN_GEO'].unique() # Obter códigos para SP dos dados auxiliares
                 # Garantir que CO_MUN no df seja numérico antes de filtrar
                 df['CO_MUN_temp'] = pd.to_numeric(df['CO_MUN'], errors='coerce')
                 df_filtered_numeric = df.dropna(subset=['CO_MUN_temp']).copy() # Trabalhar em linhas com CO_MUN numérico válido
                 df_filtered_numeric['CO_MUN_temp'] = df_filtered_numeric['CO_MUN_temp'].astype(int)
                 df_sp = df_filtered_numeric[df_filtered_numeric['CO_MUN_temp'].isin(sp_codes)].copy()
                 df_sp = df_sp.drop(columns=['CO_MUN_temp']) # Remove temporary column
                 print(f"Filtrado para SP usando 'CO_MUN': {len(df_sp)} linhas encontradas.")
            else:
                 # Se nenhum método funcionar
                 print("--- AVISO ---: Não foi possível filtrar municípios de SP.")
                 print("Motivo: coluna 'SG_UF_MUN' ausente ou inutilizável, E (coluna 'CO_MUN' ausente ou 'municipios_df' auxiliar indisponível/vazio).")
                 print("Prosseguindo com dados não filtrados para este ano, que podem incluir localizações fora de SP.")
                 df_sp = df.copy() # Usar o dataframe completo como último recurso

            if df_sp.empty:
                print(f"Nenhum dado de SP encontrado (ou filtrado) para {ano}. Pulando processamento adicional para este ano.")
                continue # Pular para o próximo ano

            # --- Converter Colunas Numéricas e Identificadores ---
            # Converter com coerção de erros, permitindo remover linhas problemáticas posteriormente
            df_sp['KG_LIQUIDO'] = pd.to_numeric(
                df_sp['KG_LIQUIDO'].str.replace(',', '.', regex=False), errors='coerce'
            )
            df_sp['VL_FOB'] = pd.to_numeric(
                df_sp['VL_FOB'].str.replace(',', '.', regex=False), errors='coerce'
            )
            # Converter identificadores chave, com coerção de erros
            df_sp['CO_MUN'] = pd.to_numeric(df_sp['CO_MUN'], errors='coerce')
            df_sp['SH4'] = pd.to_numeric(df_sp['SH4'], errors='coerce')


            # --- Remover Linhas com Dados Essenciais Inválidos ---
            # Colunas essenciais para análise: CO_MUN, SH4, KG_LIQUIDO, VL_FOB
            essential_cols = ['CO_MUN', 'SH4', 'KG_LIQUIDO', 'VL_FOB']
            initial_rows = len(df_sp)
            df_sp.dropna(subset=essential_cols, inplace=True)
            dropped_rows = initial_rows - len(df_sp)
            if dropped_rows > 0:
                print(f"Removidas {dropped_rows} linhas devido a valores essenciais ausentes/inválidos em: {', '.join(essential_cols)}.")


            # --- Converter Identificadores para Tipo Inteiro ---
            # Fazer isso *após* remover NaNs introduzidos por 'coerce'
            if not df_sp.empty:
                df_sp['CO_MUN'] = df_sp['CO_MUN'].astype(int)
                df_sp['SH4'] = df_sp['SH4'].astype(int)
            else:
                print(f"Nenhum dado válido restou para {ano} após limpeza das colunas essenciais.")
                continue # Pular para o próximo ano se a limpeza removeu todos os dados


            # --- Armazenar Dados Processados ---
            if not df_sp.empty:
                 dados[ano] = df_sp
                 print(f"Dados para {ano} processados e armazenados com sucesso. Formato final: {df_sp.shape}")
                 # Opcional: Exibir amostra/informações para verificação
                 # print(f"{ano} Amostra de Dados de {ano} (primeiras 5 linhas):\n", df_sp.head())
                 # print(f"{ano} Tipos de Dados de:\n", df_sp.dtypes)
            # Nenhum else necessário aqui, já tratado pelas declarações 'continue' acima

        except FileNotFoundError:
            print(f"--- AVISO ---: Arquivo não encontrado para o ano {ano} em {file_path}. Pulando este ano.")
        except Exception as e:
            print(f"--- ERRO ---: Ocorreu um erro inesperado ao carregar ou processar dados para o ano {ano}: {e}")
            # Opcionalmente, você pode armazenar um placeholder ou pular dependendo da gravidade
            # Por enquanto, apenas imprimimos o erro e continuamos para o próximo ano

    # --- Verificação Final ---
    if not dados:
        print("\n--- AVISO ---: Nenhum dado anual pôde ser carregado com sucesso.")
        print("Por favor, verifique os caminhos dos arquivos, existência dos arquivos e possíveis erros nos logs acima.")
    else:
        print(f"\nCarregamento de dados anuais concluído. Anos carregados com sucesso: {list(dados.keys())}")

    return dados


print("Célula 4/11: Função de carregamento de dados 'carregar_dados_anuais' definida.")

In [ ]:
# ===============================
# CÉLULA 5: Executar Carregamento de Dados & Preparar Listas
# ===============================
# Definir os anos que deseja carregar
anos_para_carregar = ['2019', '2020', '2021', '2022', '2023', '2024'] # Ajustar conforme necessário
# NOTE: '2024' podem ser parciais ou indisponíveis, adicione se tiver o arquivo.

# --- Carregar os dados usando a função ---
# Este dicionário irá conter os DataFrames, indexados por ano
dados_ano = carregar_dados_anuais(anos_para_carregar)

# --- Preparar Lista de Municípios de SP para Dropdowns ---
municipios_sp_list = ['-- N/A --'] # Placeholder padrão se o carregamento falhar
municipios_sp = pd.DataFrame() # Inicializar dataframe vazio para subconjunto SP de municipios_df

# Verificar se o carregamento de dados foi bem-sucedido e se os dados auxiliares existem
if dados_ano and 'municipios_df' in globals() and not municipios_df.empty:
    print("\nPreparando lista de municípios de SP para dropdowns...")
    # Obter todos os códigos CO_MUN únicos presentes em TODOS os anos carregados PARA SP
    all_sp_mun_codes = set()
    for ano, df_ano in dados_ano.items():
        if df_ano is not None and not df_ano.empty and 'CO_MUN' in df_ano.columns:
             all_sp_mun_codes.update(df_ano['CO_MUN'].unique())
        # else:
        #      print(f" Nota: Sem dados ou coluna CO_MUN em dados_ano['{ano}'] para extrair códigos.")

    if all_sp_mun_codes:
        print(f"Encontrados {len(all_sp_mun_codes)} códigos CO_MUN únicos de SP nos dados carregados.")
        # Filtrar o municipios_df principal baseado nesses códigos encontrados nos dados reais
        # E garantir que estão marcados como SP ('SG_UF') no arquivo auxiliar para robustez
        try:
            # Garantir que CO_MUN_GEO em municipios_df seja inteiro para correspondência
            municipios_df['CO_MUN_GEO'] = municipios_df['CO_MUN_GEO'].astype(int) # Deveria ser feito na célula 3, mas verificação segura
            municipios_sp = municipios_df[
                (municipios_df['CO_MUN_GEO'].isin(all_sp_mun_codes)) &
                (municipios_df['SG_UF'] == 'SP') # Verificar explicitamente o código de estado SP
            ].copy()

            # Criar a lista de nomes a partir do DataFrame filtrado
            if not municipios_sp.empty:
                 municipios_sp_list = sorted(municipios_sp['NO_MUN'].astype(str).unique().tolist())
                 print(f" Lista de {len(municipios_sp_list)} nomes únicos de municípios de SP encontrados nos dados criada com sucesso.")
                 if not municipios_sp_list: # Não deve acontecer se municipios_sp não estiver vazio, mas verificação de segurança
                      print(" Aviso: A filtragem resultou em uma lista vazia, embora o subconjunto SP de municipios_df não estivesse vazio. Verifique a coluna 'NO_MUN'.")
                      municipios_sp_list = ['-- N/A --']
            else:
                 print("Aviso: Nenhum município dos dados carregados correspondeu a entradas de SP no arquivo auxiliar")
                 municipios_sp_list = ['-- N/A --']

        except Exception as e:
            print(f"Erro durante a filtragem/criação da lista de municípios de SP: {e}")
            municipios_sp_list = ['-- N/A --'] # Redefinir em caso de erro
            municipios_sp = pd.DataFrame() # Redefinir dataframe SP em caso de erro

    else:
        print(" Aviso: Nenhum código de município de SP foi extraído dos dados anuais carregados. Não é possível criar lista de municípios.")
        municipios_sp_list = ['-- N/A --']
        # Garantir que municipios_sp seja um DataFrame vazio
        municipios_sp = pd.DataFrame(columns=municipios_df.columns if 'municipios_df' in globals() else [])

else:
    print("\n--- AVISO ---")
    if not dados_ano:
        print("Dados anuais ('dados_ano') não puderam ser carregados.")
    if 'municipios_df' not in globals() or municipios_df.empty:
        print("Dados auxiliares ('municipios_df') não estão disponíveis ou estão vazios.")
    print("Não é possível preparar lista de municípios de SP. Dropdowns serão limitados.")
    municipios_sp_list = ['-- N/A --']
    # EGarantir que municipios_sp seja definido, mesmo que vazio
    municipios_sp = pd.DataFrame(columns=municipios_df.columns if 'municipios_df' in globals() else [])


# Exibir primeiras opções para verificação
print(f"\nLista Final de Municípios de SP para Dropdowns (primeiros 20): {municipios_sp_list[:20]}")
if municipios_sp_list == ['-- N/A --']:
    print("-> Dropdowns mostrarão N/A. Por favor, verifique as etapas de carregamento de dados (Células 2-5).")

print("\nCélula 5/11: Carregamento de dados anuais executado, lista de SP preparada.")

In [ ]:
# ===============================
# CÉLULA 6: Definições de Widgets
# ===============================
# Definir todos os widgets interativos (dropdowns, botões, área de saída)

print("Definindo widgets...")

# --- Determinar Opções Válidas ---
# Usar chaves do dicionário de dados carregados para anos
valid_anos = list(dados_ano.keys()) if dados_ano else ['Select Year']
# Usar a lista gerada para municípios
valid_municipios = municipios_sp_list if municipios_sp_list and municipios_sp_list[0] != '-- N/A --' else ['Select Municipality']

# --- Definir Valores Padrão Sensíveis ---
default_ano = valid_anos[0] if valid_anos and valid_anos[0] != 'Select Year' else None
default_municipio1 = valid_municipios[0] if valid_municipios and valid_municipios[0] != 'Select Municipality' else None
# Tentar selecionar um segundo município *diferente* se possível
default_municipio2 = valid_municipios[1] if len(valid_municipios) > 1 and valid_municipios[0] != 'Select Municipality' else default_municipio1

# --- Criar Widgets ---

# Dropdown de Ano
ano_dropdown = widgets.Dropdown(
    options=valid_anos,
    value=default_ano,
    description='Ano:',
    disabled=not default_ano, # Desabilitar se nenhum ano válido carregado
    style={'description_width': 'initial'} # Ajustar largura para caber a descrição
)

# Dropdown de Município Único (para visualização única)
municipio_dropdown = widgets.Dropdown(
    options=valid_municipios,
    value=default_municipio1,
    description='Município:',
    disabled=not default_municipio1, # Desabilitar se nenhum município válido
    style={'description_width': 'initial'}
)

# Dropdown de Métrica
metrica_dropdown = widgets.Dropdown(
    options=['Quilogramas', 'Valor Agregado'], # As duas métricas de análise
    value='Quilogramas', # Métrica padrão
    description='Métrica:',
    style={'description_width': 'initial'}
)

# Botão para Top 5 Municípios Exportadores (geral SP)
top5_municipios_button = widgets.Button(
    description='Top 5 Municípios Exportadores (SP)',
    button_style='info', # 'info', 'success', 'warning', 'danger' or ''
    tooltip='Mostrar os 5 municípios de SP que mais exportaram no ano selecionado, pela métrica escolhida.',
    icon='list-ol', # Nome do ícone FontAwesome
    disabled=not default_ano # Desabilitar se nenhum dado carregado (nenhum ano selecionado)
)

# --- Widgets para Visualização de Comparação ---

# Dropdown Município 1 (para comparação)
municipio1_dropdown = widgets.Dropdown(
    options=valid_municipios,
    value=default_municipio1,
    description='Comparar Município 1:',
    disabled=not default_municipio1,
    style={'description_width': 'initial'}
)

# Dropdown Município 2 (para comparação)
municipio2_dropdown = widgets.Dropdown(
    options=valid_municipios,
    value=default_municipio2, # Usar padrão potencialmente diferente
    description='Comparar Município 2:',
    disabled=not default_municipio2, # Verificar o segundo padrão especificamente
    style={'description_width': 'initial'}
)

# Botão para acionar gráfico de comparação
compare_button = widgets.Button(
    description='Comparar Top 5 Produtos',
    button_style='success',
    tooltip='Comparar os top 5 produtos (pela métrica) dos dois municípios selecionados.',
    icon='exchange-alt', # Nome do ícone FontAwesome
    disabled=not default_municipio1 or not default_municipio2 #  Desabilitar se qualquer padrão for inválido/indisponível
)

# --- Widget de Área de Saída ---
# Este widget irá conter os gráficos e mensagens gerados pelos manipuladores
output_area = widgets.Output(layout={'border': '1px solid black', 'padding': '10px'}) # Adicionar borda para visibilidade

print("Widgets definidos com sucesso.")
print("--> Os widgets serão exibidos na Célula 11 após a configuração dos manipuladores de eventos.")

print("\nCélula 6/11: Widgets definidos.")

In [ ]:
# ===============================
# CÉLULA 7: Funções Auxiliares & de Cálculo
# ===============================
# Definir funções usadas para pesquisa de dados e cálculos de métricas

# --- Função auxiliar para obter CO_MUN_GEO a partir de NO_MUN ---
def get_co_mun(nome_municipio):
    """Procura o código CO_MUN_GEO para um nome de município usando o DataFrame 'municipios_sp."""
    # Verificar se o dataframe necessário está disponível
    if 'municipios_sp' not in globals() or municipios_sp.empty:
         print(f"Aviso: DataFrame 'municipios_sp' não está disponível ou está vazio. Não é possível procurar CO_MUN para '{nome_municipio}'.")
         return None

    try:
        # Garantir tipos de dados consistentes para correspondência
        municipios_sp['NO_MUN'] = municipios_sp['NO_MUN'].astype(str)
        nome_municipio_str = str(nome_municipio) # Garantir que a entrada seja string

        # Executar a pesquisa
        result = municipios_sp[municipios_sp['NO_MUN'] == nome_municipio_str]['CO_MUN_GEO']

        if not result.empty:
            # Retornar a primeira correspondência se encontrada (deve ser única em municipios_sp)
            return int(result.iloc[0]) # Retornar como inteiro
        else:
            # Se não encontrado na lista filtrada de SP, registrar um erro/aviso
            print(f"Aviso: Código CO_MUN não encontrado para o município '{nome_municipio_str}' na lista filtrada 'municipios_sp'.")
            # Opcional: Você poderia adicionar uma pesquisa alternativa no 'municipios_df' principal aqui se necessário,
            # mas pode indicar um problema com a lógica de filtragem de SP na Célula 5.
            return None # Indicar falha claramente
    except Exception as e:
        print(f"Erro durante a pesquisa de CO_MUN para '{nome_municipio}': {e}")
        return None


# --- Função de cálculo para os top 5 produtos de um único município ---
def calcular_top_produtos(df_ano, co_mun, metrica):
    """
    Calcula os top 5 produtos para um município específico com base na métrica selecionada.

    Argumentos:
        df_ano (pd.DataFrame): O DataFrame contendo dados de exportação para um ano específico.
        co_mun (int): O código CO_MUN_GEO do município.
        metrica (str): A métrica a ser usada ('Quilogramas' ou 'Valor Agregado').

    Devoluções:
        pd.DataFrame: Um DataFrame com os top 5 produtos, seus valores de métrica,
                      e descrições, ou um DataFrame vazio se os dados forem insuficientes ou ocorrerem erros.
    """
    # --- Validação de Entrada ---
    if df_ano is None or df_ano.empty:
        # print(f"Debug: DataFrame de entrada para o ano está vazio ou None para CO_MUN {co_mun}.\") # Informações de depuração
        return pd.DataFrame()

    if co_mun is None:
        print("Aviso: CO_MUN inválido (None) fornecido para calcular_top_produtos.")
        return pd.DataFrame()

    # Verificar se os dados auxiliares NCM/SH estão disponíveis
    if 'ncm_sh_df' not in globals() or ncm_sh_df.empty:
        print("Error: ncm_sh_df (descrições de produtos) não carregado. Não é possível calcular os produtos principais.")
        return pd.DataFrame()

    # --- Filtrar dados para o município específico ---
    df_mun = df_ano[df_ano['CO_MUN'] == co_mun].copy()

    if df_mun.empty:
        # print(f"Debug: Nenhum dado encontrado em df_ano para CO_MUN {co_mun}.\") # Depuração mais detalhada se necessário
        return pd.DataFrame() # Retornar vazio se nenhuma linha de dados para este município no df do ano

    # --- Calcular Métrica com Base na Seleção ---
    calculated_metric_col = None # Armazenar o nome da coluna que contém o valor da métrica calculada
    df_metricas = pd.DataFrame() # Inicializar DataFrame de resultado

    if metrica == 'Quilogramas':
        coluna_orig = 'KG_LIQUIDO'
        if coluna_orig not in df_mun.columns:
             print(f"Erro: Coluna necessária '{coluna_orig}' não encontrada nos dados para CO_MUN {co_mun}.")
             return pd.DataFrame() # Não pode prosseguir
        # Agrupar por código do produto (SH4) e somar os quilogramas
        df_metricas = df_mun.groupby('SH4')[coluna_orig].sum().reset_index()
        calculated_metric_col = coluna_orig # A coluna para classificar é KG_LIQUIDO em si

    elif metrica == 'Valor Agregado':
        # Verificar colunas necessárias
        if not all(col in df_mun.columns for col in ['VL_FOB', 'KG_LIQUIDO']):
             print(f"Erro: Colunas necessárias 'VL_FOB' ou 'KG_LIQUIDO' não encontradas para CO_MUN {co_mun}.")
             return pd.DataFrame() # Não pode prosseguir

        # Agrupar por código do produto (SH4) e somar valor FOB e quilogramas
        df_metricas_agg = df_mun.groupby('SH4').agg(
            Total_FOB=('VL_FOB', 'sum'),
            Total_KG=('KG_LIQUIDO', 'sum')
        ).reset_index()

        # Filtrar linhas onde Total_KG é zero ou muito próximo de zero para evitar erros de divisão
        epsilon = 1e-9 # Usar um número muito pequeno para lidar com problemas potenciais de ponto flutuante
        df_metricas = df_metricas_agg[df_metricas_agg['Total_KG'] > epsilon].copy()

        if df_metricas.empty:
            # print(f"Debug: Nenhum produto com KG diferente de zero encontrado para CO_MUN {co_mun} para calcular Valor Agregado.") # Informação de depuração
            return pd.DataFrame() # Retornar vazio se nenhum dado válido após filtragem

        # Calcular o Valor Agregado (FOB / KG)
        df_metricas['Valor_Agregado'] = df_metricas['Total_FOB'] / df_metricas['Total_KG']
        calculated_metric_col = 'Valor_Agregado' # Esta é a coluna para classificar
    else:
        # Lidar com entrada de métrica desconhecida
        print(f"Erro: Métrica desconhecida '{metrica}' fornecida para calcular_top_produtos.")
        return pd.DataFrame()


    # --- Obter Top 5 Produtos ---
    # Verificar se o cálculo foi bem-sucedido e se a coluna da métrica existe
    if calculated_metric_col is None or calculated_metric_col not in df_metricas.columns:
         print(f"Erro: Coluna de métrica '{calculated_metric_col}' não foi calculada corretamente ou está ausente para CO_MUN {co_mun}.")
         return pd.DataFrame()

    # Garantir que há dados para classificar
    if df_metricas.empty:
        # Este caso provavelmente já foi tratado acima (ex: nenhum KG diferente de zero), mas boa medida de segurança
        return pd.DataFrame()

    # Obter as 5 principais linhas com base na coluna de métrica calculada
    top5 = df_metricas.nlargest(5, calculated_metric_col)

    # --- Adicionar Descrições dos Produtos ---
    try:
        # Garantir que as chaves de junção (códigos SH4) sejam de tipos inteiros compatíveis
        top5['SH4'] = top5['SH4'].astype(int)
        # ncm_sh_df['CO_SH4'] já deve ser int da limpeza na Célula 3
        if 'CO_SH4' not in ncm_sh_df.columns:
            print("Erro: Coluna 'CO_SH4' ausente em ncm_sh_df. Não é possível mesclar nomes de produtos.")
            top5['NO_SH4_POR'] = 'Erro no Merge (Sem CO_SH4)'
        else:
             ncm_sh_df['CO_SH4'] = ncm_sh_df['CO_SH4'].astype(int) # Garantir tipo antes do merge
             top5 = top5.merge(
                 ncm_sh_df[['CO_SH4', 'NO_SH4_POR']], # Selecionar apenas colunas necessárias de ncm_sh_df
                 left_on='SH4',
                 right_on='CO_SH4',
                 how='left' #Usar merge 'left' para manter todos os produtos top 5 mesmo se nenhuma descrição for encontrada
             )
    except Exception as e:
        print(f"Erro ao mesclar dados top 5 com descrições NCM SH para CO_MUN {co_mun}: {e}")
        top5['NO_SH4_POR'] = 'Erro no Merge' # Adicionar nome placeholder se o merge falhar

    # --- Formatação Final ---
    # Criar um rótulo 'Produto' descritivo (Descrição (Código SH4))
    # HLidar com casos onde o merge pode ter falhado ou a descrição está ausente
    top5['NO_SH4_POR'] = top5['NO_SH4_POR'].fillna('Descrição Desconhecida')
    # Garantir que SH4 seja string para concatenação, truncar descrição
    top5['Produto'] = top5['NO_SH4_POR'].astype(str).str[:40] + ' (' + top5['SH4'].astype(str) + ')'
    # Adicionar o código do município de volta para referência, se necessário posteriormente
    top5['CO_MUN'] = co_mun

    # Retornar apenas as colunas necessárias para plotagem/exibição
    # Incluir a coluna de métrica específica usada para classificação
    columns_to_return = ['CO_MUN', 'SH4', 'Produto', calculated_metric_col]
    # Adicionar outras colunas potencialmente úteis se existirem da agregação (como Total_FOB, Total_KG para Valor Agregado)
    if metrica == 'Valor Agregado' and all(c in top5.columns for c in ['Total_FOB', 'Total_KG']):
        columns_to_return.extend(['Total_FOB', 'Total_KG'])

    return top5[columns_to_return]


print("Célula 7/11: Funções auxiliares e de cálculo definidas.")

In [ ]:
# ===============================
# CÉLULA 8: Funções de Plotagem (Corrigidas)
# ===============================
# Funções para gerar os diferentes tipos de gráficos usando Matplotlib e Seaborn

# --- Plotar Top 5 Produtos para um ÚNICO Município ---
def criar_grafico_municipio_unico(dados, municipio_nome, ano, metrica):
    """Gera e exibe um gráfico de barras horizontais para os top 5 produtos de um único município."""
    print(f"--- Gerando Gráfico: Município Único ---")
    print(f"  Município: {municipio_nome}")
    print(f"  Ano: {ano}")
    print(f"  Métrica: {metrica}")

    # --- Obter Código do Município ---
    co_mun = get_co_mun(municipio_nome)
    if co_mun is None:
        # Mensagem de erro tratada por get_co_mun, apenas exibir gráfico placeholder
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, f"Erro: Código não encontrado para o município '{municipio_nome}'.",
                 ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # --- Validar Ano e Disponibilidade de Dados ---
    if ano not in dados or dados[ano] is None or dados[ano].empty:
        errmsg = f"Erro: Dados não disponíveis para o ano {ano}."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # --- Calcular Top 5 Produtos ---
    df_ano = dados[ano]
    top5_data = calcular_top_produtos(df_ano, co_mun, metrica) # Usar a função de cálculo da Célula 7

    # --- Lidar com Caso Sem Dados ---
    if top5_data.empty:
        infomsg = f"Sem dados de exportação Top 5 encontrados para:\n" \
                  f"Município: {municipio_nome} (Código: {co_mun})\n" \
                  f"Ano: {ano}\nMétrica: {metrica}"
        # FIX: Realizar substituição antes do f-string
        log_msg = infomsg.replace('\n', ' ')
        print(f"  Info: {log_msg}") # Registrar informação
        plt.figure(figsize=(10, 4)) # Tornar a figura um pouco maior para texto multi-linha
        plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
        plt.axis('off') # Ocultar eixos para exibição de mensagem
        plt.show()
        return

    # --- Preparar para Plotagem ---
    # Determinar o nome da coluna que contém o valor da métrica com base na seleção
    coluna_plot = 'KG_LIQUIDO' if metrica == 'Quilogramas' else 'Valor_Agregado'
    # Verificar novamente se esta coluna realmente existe no resultado (deveria sempre, dado calcular_top_produtos)
    if coluna_plot not in top5_data.columns:
        errmsg = f"Erro Interno: Coluna da métrica '{coluna_plot}' não encontrada nos dados calculados."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # Definir rótulos do gráfico com base na métrica
    eixo_x_label = 'Massa Líquida (Kg)' if metrica == 'Quilogramas' else 'Valor Agregado (USD/kg)'
    titulo = f'Top 5 Produtos Exportados - {municipio_nome} ({ano})'

    # --- Criar o Gráfico ---
    plt.figure(figsize=(12, 7)) # Ajustar tamanho da figura conforme necessário para legibilidade

    # Ordenar dados para ordem de plotagem consistente (ex: barra com maior valor no topo)
    # Para barra horizontal (barh), ordenar em ordem crescente significa que o maior valor fica no topo
    plot_data = top5_data.sort_values(coluna_plot, ascending=True)

    # Criar o gráfico de barras horizontais usando Seaborn
    sns.barplot(data=plot_data, y='Produto', x=coluna_plot, color='cornflowerblue', orient='h') # Cor alterada

    # Personalizar aparência do gráfico
    plt.title(titulo, fontsize=16, pad=15) # Adicionado padding
    plt.xlabel(eixo_x_label, fontsize=13)
    plt.ylabel('Produto (Descrição Resumida e Código SH4)', fontsize=13) # Rótulo do eixo y mais descritivo
    plt.xticks(rotation=45, ha='right', fontsize=11) # Rotacionar rótulos do eixo x se valores grandes
    plt.yticks(fontsize=11)
    plt.grid(axis='x', linestyle='--', alpha=0.6) # Adicionar linhas de grade horizontais
    plt.tight_layout() # Ajustar layout automaticamente para evitar sobreposição de rótulos
    plt.show() # Exibir o gráfico
    print(f"  Plot displayed successfully for {municipio_nome}.")


# --- Plotar Top 5 Municípios Exportadores em SP ---
def criar_grafico_top5_municipios(dados, ano, metrica):
    """Gera e exibe um gráfico de barras para os top 5 municípios exportadores de SP com base na métrica selecionada."""
    print(f"--- Gerando Gráfico: Top 5 Municípios SP ---")
    print(f"  Ano: {ano}")
    print(f"  Métrica: {metrica}")

    # --- Validar Ano e Disponibilidade de Dados ---
    if ano not in dados or dados[ano] is None or dados[ano].empty:
        errmsg = f"Erro: Dados não disponíveis para o ano {ano}."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    df_ano = dados[ano]

    # --- Verificar se Dados de Nomes de Municípios de SP Estão Disponíveis ---
    if 'municipios_sp' not in globals() or municipios_sp.empty:
        errmsg = "Erro: Dados auxiliares de municípios ('municipios_sp') não disponíveis ou vazios.\n" \
                 "Não é possível adicionar nomes de municípios ao gráfico."
        # FIX: Realizar substituição antes do f-string
        log_msg = errmsg.replace('\n', ' ')
        print(f"  {log_msg}")
        plt.figure(figsize=(10, 3))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return # Parar, pois nomes são essenciais para este gráfico

    # --- Agregar Dados por Município com Base na Métrica ---
    df_agregado = pd.DataFrame() # Inicializar DataFrame de resultado
    coluna_rank = None # Isso irá conter o nome da coluna usada para classificação
    eixo_x_label = '' # Rótulo do eixo x do gráfico

    if metrica == 'Quilogramas':
        coluna_orig = 'KG_LIQUIDO'
        if coluna_orig not in df_ano.columns:
             print(f"  Erro: Coluna necessária '{coluna_orig}' não encontrada nos dados para {ano}.")
             return # Parar se coluna essencial estiver ausente
        # Agrupar por município e somar os quilogramas
        df_agregado = df_ano.groupby('CO_MUN')[coluna_orig].sum().reset_index()
        coluna_rank = coluna_orig # Classificar pela soma de quilogramas
        eixo_x_label = 'Total Exportado (Kg)'

    elif metrica == 'Valor Agregado':
        # Calcular Valor Agregado médio geral para cada município no ano
        if not all(col in df_ano.columns for col in ['VL_FOB', 'KG_LIQUIDO', 'CO_MUN']):
             print(f"  Erro: Colunas necessárias ('VL_FOB', 'KG_LIQUIDO', 'CO_MUN') não encontradas para {ano}.")
             return # Parar se colunas essenciais estiverem ausentes

        # Agrupar por município, somando FOB e KG
        df_agregado_temp = df_ano.groupby('CO_MUN').agg(
            Total_FOB=('VL_FOB', 'sum'),
            Total_KG=('KG_LIQUIDO', 'sum')
        ).reset_index()

        # Filtrar municípios com total KG zero ou próximo de zero para evitar erros de divisão
        epsilon = 1e-9
        df_agregado = df_agregado_temp[df_agregado_temp['Total_KG'] > epsilon].copy()

        if df_agregado.empty:
            infomsg = f"Sem municípios com exportações válidas (KG > 0) encontradas em {ano}\n" \
                      f"para calcular o Valor Agregado médio."
            # FIX: Realizar substituição antes do f-string
            log_msg = infomsg.replace('\n', ' ')
            print(f"  Info: {log_msg}")
            plt.figure(figsize=(10, 4))
            plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
            plt.axis('off')
            plt.show()
            return

        # CCalcular o Valor Agregado médio para cada município
        df_agregado['Valor_Agregado_Medio'] = df_agregado['Total_FOB'] / df_agregado['Total_KG']
        coluna_rank = 'Valor_Agregado_Medio' # Classificar por esta média calculada
        eixo_x_label = 'Valor Agregado Médio Municipal (USD/kg)'
    else:
        # Lidar com métrica desconhecida
        print(f"  Erro: Métrica desconhecida '{metrica}' para o gráfico Top 5 Municípios.")
        return


    # --- Lidar com métrica desconhecida ---
    # Verificar se a agregação e determinação da coluna de classificação foram bem-sucedidas
    if coluna_rank is None or coluna_rank not in df_agregado.columns:
         print(f"  Erro Interno: Coluna de ranking '{coluna_rank}' não encontrada nos dados agregados.")
         return

    # Obter os 5 principais municípios com base na coluna de classificação
    top5_mun_codes = df_agregado.nlargest(5, coluna_rank)

    if top5_mun_codes.empty:
        infomsg = f"Não foi possível encontrar municípios para o ranking\n" \
                  f"Métrica: {metrica}\nAno: {ano}"
        # FIX: Realizar substituição antes do f-string
        log_msg = infomsg.replace('\n', ' ')
        print(f"  Info: {log_msg}")
        plt.figure(figsize=(10, 4))
        plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
        plt.axis('off')
        plt.show()
        return

    # --- Mesclar com Nomes de Municípios ---
    try:
         # Garantir que as chaves de junção ('CO_MUN' e 'CO_MUN_GEO') sejam tipos inteiros compatíveis
         top5_mun_codes['CO_MUN'] = top5_mun_codes['CO_MUN'].astype(int)
         # municipios_sp['CO_MUN_GEO'] já deve ser int da limpeza na Célula 5
         if 'CO_MUN_GEO' not in municipios_sp.columns:
             print("Erro: Coluna 'CO_MUN_GEO' ausente em municipios_sp. Não é possível mesclar nomes")
             top5_mun_data = top5_mun_codes.copy()
             top5_mun_data['NO_MUN'] = 'Erro no Merge (Sem CO_MUN_GEO)'
         else:
             municipios_sp['CO_MUN_GEO'] = municipios_sp['CO_MUN_GEO'].astype(int) # Garantir tipo
             top5_mun_data = top5_mun_codes.merge(
                 municipios_sp[['CO_MUN_GEO', 'NO_MUN']], # Selecionar apenas colunas necessárias
                 left_on='CO_MUN',
                 right_on='CO_MUN_GEO',
                 how='left' # Manter todos os códigos top 5, mesmo se a junção de nomes falhar (indica problema nos dados)
             )
    except Exception as e:
         print(f"Erro ao mesclar códigos de municípios Top 5 com nomes: {e}")
         # Criar uma coluna de nome placeholder se a junção falhar para permitir plotagem de códigos
         top5_mun_data = top5_mun_codes.copy()
         top5_mun_data['NO_MUN'] = 'Erro no Merge - Código: ' + top5_mun_data['CO_MUN'].astype(str)

    # Lidar com nomes ausentes (se a junção falhou para alguns códigos)
    top5_mun_data['NO_MUN'] = top5_mun_data['NO_MUN'].fillna('Nome Desconhecido')

    # --- Criar o Gráfico ---
    plt.figure(figsize=(11, 7)) # Tamanho ajustado

    # Ordenar dados para ordem de plotagem consistente (barra com maior valor no topo)
    plot_data = top5_mun_data.sort_values(coluna_rank, ascending=True)

    # Criar o gráfico de barras horizontais usando Seaborn
    sns.barplot(data=plot_data, y='NO_MUN', x=coluna_rank, color='mediumseagreen', orient='h') # Cor alterada

    # Personalizar aparência do gráfico
    plt.title(f'Top 5 Municípios Exportadores SP ({ano}) por {metrica}', fontsize=16, pad=15)
    plt.xlabel(eixo_x_label, fontsize=13)
    plt.ylabel('Município', fontsize=13)
    plt.xticks(rotation=45, ha='right', fontsize=11)
    plt.yticks(fontsize=11)
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show() # Exibir o gráfico
    print(f"Gráfico exibido com sucesso para Top 5 Municípios.")


# --- Plotar Comparação entre DOIS Municípios ---
def criar_grafico_comparativo(dados, ano, metrica, municipio1_nome, municipio2_nome):
    """
    Gera e exibe um gráfico de barras horizontais agrupado comparando os top 5 produtos
    (com base na união dos top 5 individuais) de dois municípios selecionados.
    """
    print(f"--- Gerando Gráfico: Comparação de Municípios ---")
    print(f"  Município 1: {municipio1_nome}")
    print(f"  Município 2: {municipio2_nome}")
    print(f"  Ano: {ano}")
    print(f"  Métrica: {metrica}")

    # --- Validação de Entrada ---
    if not municipio1_nome or not municipio2_nome or municipio1_nome == '-- N/A --' or municipio2_nome == '-- N/A --' or municipio1_nome == 'Select Municipality' or municipio2_nome == 'Select Municipality':
         errmsg = "Erro: Por favor, selecione dois municípios válidos para comparação."
         print(f"  {errmsg}")
         plt.figure(figsize=(10, 2))
         plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
         plt.axis('off')
         plt.show()
         return
    if municipio1_nome == municipio2_nome:
        errmsg = "Erro: Por favor, selecione dois municípios diferentes para comparação."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # --- Obter Códigos dos Municípios ---
    co_mun1 = get_co_mun(municipio1_nome)
    co_mun2 = get_co_mun(municipio2_nome)
    if co_mun1 is None or co_mun2 is None:
        # Mensagem de erro tratada por get_co_mun, criar gráfico placeholder
        missing = []
        if co_mun1 is None: missing.append(f"'{municipio1_nome}'")
        if co_mun2 is None: missing.append(f"'{municipio2_nome}'")
        errmsg = f"Erro: Código não encontrado para: {', '.join(missing)}."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # --- Validar Ano e Disponibilidade de Dados ---
    if ano not in dados or dados[ano] is None or dados[ano].empty:
        errmsg = f"Erro: Dados não disponíveis para o ano {ano}."
        print(f"  {errmsg}")
        plt.figure(figsize=(10, 2))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # --- Verificar se Dados de Descrição de Produtos Estão Disponíveis ---
    if 'ncm_sh_df' not in globals() or ncm_sh_df.empty:
        errmsg = "Erro: Dados auxiliares de produtos ('ncm_sh_df') não disponíveis.\n" \
                 "Não é possível adicionar descrições de produtos ao gráfico."
        # FIX: Realizar substituição antes do f-string
        log_msg = errmsg.replace('\n', ' ')
        print(f"  {log_msg}")
        plt.figure(figsize=(10, 3))
        plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='red')
        plt.axis('off')
        plt.show()
        return

    # --- Calcular Métricas para TODOS os Produtos para Ambos os Municípios ---
    df_ano = dados[ano]
    metrica_col_calc = 'KG_LIQUIDO' if metrica == 'Quilogramas' else 'Valor_Agregado' # Nome da coluna de métrica calculada
    eixo_x_label = 'Massa Líquida (Kg)' if metrica == 'Quilogramas' else 'Valor Agregado (USD/kg)'

    # Função auxiliar para calcular métrica para todos os produtos de um município
    def calculate_all_products(df_year, co_mun_sel, metric_name):
        # Filtrar para o município
        df_mun_all = df_year[df_year['CO_MUN'] == co_mun_sel].copy()
        if df_mun_all.empty: return pd.DataFrame() # Retornar vazio se nenhuma linha de dados

        metric_col = None # Irá conter o nome da coluna de métrica calculada
        agg_df = pd.DataFrame() # Inicializar

        if metric_name == 'Quilogramas':
             if 'KG_LIQUIDO' not in df_mun_all.columns: return pd.DataFrame() # Verificar coluna necessária
             agg_df = df_mun_all.groupby('SH4')['KG_LIQUIDO'].sum().reset_index()
             metric_col = 'KG_LIQUIDO'
        elif metric_name == 'Valor Agregado':
             if not all(c in df_mun_all.columns for c in ['VL_FOB', 'KG_LIQUIDO']): return pd.DataFrame() # Verificar colunas
             agg_tmp = df_mun_all.groupby('SH4').agg(Total_FOB=('VL_FOB', 'sum'), Total_KG=('KG_LIQUIDO', 'sum')).reset_index()
             epsilon = 1e-9 # Usar epsilon pequeno para comparação
             agg_df = agg_tmp[agg_tmp['Total_KG'] > epsilon].copy() # Filtrar KG não-zero
             if agg_df.empty: return pd.DataFrame() # Retornar vazio se nenhuma linha válida restar
             agg_df['Valor_Agregado'] = agg_df['Total_FOB'] / agg_df['Total_KG'] # Calcular métrica
             metric_col = 'Valor_Agregado'
        else:
             print(f"Erro: Métrica desconhecida '{metric_name}' na função interna.")
             return pd.DataFrame() # Métrica inválida

        # Retornar DataFrame com SH4 e a coluna de métrica calculada, se bem-sucedido
        if metric_col and metric_col in agg_df.columns and not agg_df.empty:
             return agg_df[['SH4', metric_col]].copy() # Retornar apenas SH4 e o valor da métrica
        else:
             return pd.DataFrame() # Retornar vazio em caso de falha

    # Calcular para ambos os municípios
    mun1_agg = calculate_all_products(df_ano, co_mun1, metrica)
    mun2_agg = calculate_all_products(df_ano, co_mun2, metrica)

    # --- Verificar se Dados Foram Encontrados para Cálculo ---
    valid_mun1 = not mun1_agg.empty
    valid_mun2 = not mun2_agg.empty

    # Lidar com casos onde um ou ambos os municípios não têm dados válidos para a métrica
    if not valid_mun1 and not valid_mun2:
        infomsg = f"Sem dados válidos para a métrica '{metrica}' ({ano})\n" \
                  f"encontrados para NENHUM dos municípios:\n" \
                  f"{municipio1_nome} ou {municipio2_nome}."
        # FIX: Realizar substituição antes do f-string
        log_msg = infomsg.replace('\n', ' ')
        print(f"  Info: {log_msg}")
        plt.figure(figsize=(10, 4))
        plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
        plt.axis('off')
        plt.show()
        return
    elif not valid_mun1:
         print(f"Aviso: Sem dados válidos encontrados para {municipio1_nome} (Métrica: {metrica}, Ano: {ano}). Comparação pode estar incompleta.")
    elif not valid_mun2:
         print(f"Aviso: Sem dados válidos encontrados para {municipio2_nome} (Métrica: {metrica}, Ano: {ano}). Comparação pode estar incompleta.")


    # --- Identificar Top 5 Produtos (Códigos SH4) para Cada Município ---
    # Usar os dados agregados (mun1_agg, mun2_agg) que contêm a coluna de métrica calculada
    top5_sh4_mun1 = set(mun1_agg.nlargest(5, metrica_col_calc)['SH4']) if valid_mun1 else set()
    top5_sh4_mun2 = set(mun2_agg.nlargest(5, metrica_col_calc)['SH4']) if valid_mun2 else set()

    # --- União de Produtos Principais ---
    # Encontrar todos os códigos SH4 únicos que aparecem no top 5 de QUALQUER município
    union_sh4 = top5_sh4_mun1.union(top5_sh4_mun2)

    if not union_sh4:
        infomsg = f"Nenhum produto comum no Top 5 encontrado para\n" \
                  f"{municipio1_nome} e {municipio2_nome} ({ano})\n" \
                  f"Métrica: {metrica}"
        # FIX: Realizar substituição antes do f-string
        log_msg = infomsg.replace('\n', ' ')
        print(f"  Info: {log_msg}")
        plt.figure(figsize=(10, 4))
        plt.text(0.5, 0.5, infomsg, ha='center', va='center', fontsize=12, color='blue')
        plt.axis('off')
        plt.show()
        return

    print(f"Comparando com base em {len(union_sh4)} produto(s) únicos do Top 5 (Códigos SH4: {union_sh4})")

    # --- Preparar Dados para Plotagem ---
    # Filtrar os dados agregados para AMBOS os municípios para incluir APENAS os produtos no conjunto união.
    # Adicionar uma coluna 'Municipio' para distingui-los.

    # Filtrar Dados do Munícipio 1 
    if valid_mun1:
        mun1_plot_data = mun1_agg[mun1_agg['SH4'].isin(union_sh4)].copy()
        mun1_plot_data['Municipio'] = municipio1_nome
    else:
        # Criar estrutura placeholder se não houver dados para o município 1
        mun1_plot_data = pd.DataFrame({'SH4': list(union_sh4), metrica_col_calc: 0, 'Municipio': municipio1_nome})


    # Filtrar Dados do Munícipio 2  
    if valid_mun2:
        mun2_plot_data = mun2_agg[mun2_agg['SH4'].isin(union_sh4)].copy()
        mun2_plot_data['Municipio'] = municipio2_nome
    else:
         # Criar estrutura placeholder se não houver dados para o município 2
        mun2_plot_data = pd.DataFrame({'SH4': list(union_sh4), metrica_col_calc: 0, 'Municipio': municipio2_nome})


    # --- Combinar Dados para Plotagem ---
    # Concatenar os dois dataframes
    combined_data = pd.concat([mun1_plot_data, mun2_plot_data], ignore_index=True)

    # --- Adicionar Descrições dos Produtos ---
    try:
        # Garantir que as chaves de junção sejam tipos compatíveis
        combined_data['SH4'] = combined_data['SH4'].astype(int)
        # ncm_sh_df['CO_SH4'] deve ser int da Célula 3
        if 'CO_SH4' not in ncm_sh_df.columns:
             print("Erro: Coluna 'CO_SH4' ausente em ncm_sh_df. Não é possível mesclar nomes de produtos.")
             combined_data['NO_SH4_POR'] = 'Erro no Merge (Sem CO_SH4)'
        else:
             ncm_sh_df['CO_SH4'] = ncm_sh_df['CO_SH4'].astype(int) # Garantir tipo
             combined_data = combined_data.merge(
                 ncm_sh_df[['CO_SH4', 'NO_SH4_POR']],
                 left_on='SH4',
                 right_on='CO_SH4',
                 how='left' # Manter todos os produtos mesmo se a descrição estiver ausente
             )
    except Exception as e:
        print(f"Erro ao mesclar dados combinados com nomes NCM SH: {e}")
        combined_data['NO_SH4_POR'] = 'Erro no Merge' # Placeholder

    # Lidar com descrições ausentes e criar rótulo 'Produto' final
    combined_data['NO_SH4_POR'] = combined_data['NO_SH4_POR'].fillna('Descrição Desconhecida')
    combined_data['Produto'] = combined_data['NO_SH4_POR'].astype(str).str[:40] + ' (' + combined_data['SH4'].astype(str) + ')'

    # Preencher valores de métrica ausentes com 0 APÓS o merge (ex: se um produto foi top 5 para um município mas não exportado pelo outro)
    combined_data[metrica_col_calc] = combined_data[metrica_col_calc].fillna(0)


    # --- Criar o Gráfico de Barras Agrupado ---
    plt.figure(figsize=(14, max(6, len(union_sh4) * 0.9))) # Altura dinâmica baseada no número de produtos

    # Definir uma paleta de cores 
    palette = {'viridis': sns.color_palette("viridis", n_colors=2),
               'paired': sns.color_palette("Paired", n_colors=2)}
    plot_palette = palette['viridis'] # Escolher uma

    # --- Opcional: Ordenar Produtos para Clareza do Gráfico ---
    # Ordenar pelo valor máximo alcançado por qualquer município para aquele produto
    try:
        if not combined_data.empty and metrica_col_calc in combined_data.columns:
            # Agrupar por produto, encontrar o valor máximo entre os municípios, ordenar produtos por este valor máximo
            product_order = combined_data.groupby('Produto')[metrica_col_calc].max().sort_values(ascending=False).index
            print(f"Ordem dos produtos no gráfico (decrescente por valor máximo): {list(product_order)}")
        else:
            product_order = None # Nenhuma ordem específica se o cálculo falhar
    except Exception as e:
        print(f"Aviso: Não foi possível determinar a ordem dos produtos para o gráfico. Usando ordem padrão. Erro: {e}")
        product_order = None


    # Criar o gráfico de barras horizontais agrupado usando Seaborn
    sns.barplot(
        data=combined_data,
        y='Produto',
        x=metrica_col_calc,
        hue='Municipio', # Isso cria o agrupamento
        orient='h',
        order=product_order, # Aplicar a ordem calculada (ou None)  
        palette=plot_palette # Usar paleta definida
    )

    # Personalizar aparência do gráfico  
    plt.title(f'Comparação Top Produtos ({metrica}) - {municipio1_nome} vs {municipio2_nome} ({ano})', fontsize=16, pad=15)
    plt.xlabel(eixo_x_label, fontsize=13)
    plt.ylabel('Produto (Descrição Resumida e Código SH4)', fontsize=13) # Rótulo do eixo y mais des
    plt.xticks(rotation=45, ha='right', fontsize=11)
    plt.yticks(fontsize=11)
    plt.legend(title='Município', fontsize=11, title_fontsize=12, loc='lower right') # Ajustar legenda
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show() # Exibir o gráfico
    print(f"Gráfico de comparação exibido com sucesso.")


print("Célula 8/11: Funções de plotagem definidas (Corrigidas).")

In [ ]:
# ===============================
# CÉLULA 9: Manipuladores de Eventos
# ===============================
# Definir funções que serão chamadas quando os valores dos widgets mudarem ou botões forem clicados.
# Essas funções normalmente limparão a área de saída e chamarão a função de plotagem apropriada
print("Definindo funções de manipulador de eventos...")

# --- Manipulador para Mudanças nos Dropdowns (Ano, Município Único, Métrica) ---
def on_dropdown_change(change):
    """
    Manipula mudanças no 'value' de ano_dropdown, municipio_dropdown ou metrica_dropdown.
    Atualiza o gráfico de município único na output_area.
    """
    # Verificar se a notificação de mudança é para o atributo 'value'
    if change['type'] == 'change' and change['name'] == 'value':
        # Usar o gerenciador de contexto 'output_area' para direcionar saída (gráficos, prints)
        with output_area:
            clear_output(wait=True) # Limpar gráfico/mensagens anteriores antes de gerar novo
            print(">>> Evento: Dropdown alterado. Atualizando gráfico de município único...") # Feedback para usuário

            # Obter os valores atuais dos dropdowns relevantes
            selected_ano = ano_dropdown.value
            selected_municipio = municipio_dropdown.value # Este é o do gráfico único
            selected_metrica = metrica_dropdown.value

            # --- Validar Seleções Antes de Plotar ---
            if not selected_ano or not selected_municipio or \
               selected_municipio in ['-- N/A --', 'Selecionar Município']:
                errmsg = "Por favor, selecione um Ano e um Município válidos para visualizar o gráfico."
                print(f"! Erro de Validação: {errmsg}")
                # Exibir mensagem na área de gráfico em vez de um gráfico
                plt.figure(figsize=(10, 2))
                plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='orange')
                plt.axis('off')
                plt.show()
                return # Parar execução para este manipulador

            # --- Chamar a Função de Plotagem ---
            # Passar o dicionário global de dados e valores atuais dos widgets
            criar_grafico_municipio_unico(
                dados=dados_ano,
                municipio_nome=selected_municipio,
                ano=selected_ano,
                metrica=selected_metrica
            )
            print("<<< Atualização de gráfico concluída.")


# --- Manipulador para Clique no Botão Top 5 Municípios SP ---
def on_top5_municipios_clicked(button_info): # O próprio objeto do botão é passado
     """Manipula cliques no botão 'Top 5 Municípios Exportadores (SP)."""
     # Usar o gerenciador de contexto 'output_area'
     with output_area:
        clear_output(wait=True) # Limpar saída anterior
        print(">>> Evento: Botão 'Top 5 Municípios' clicado. Gerando gráfico") # Feedback para usuário

        # Obter valores atuais dos filtros comuns
        selected_ano = ano_dropdown.value
        selected_metrica = metrica_dropdown.value

        # --- Validar Seleções ---
        if not selected_ano:
             errmsg = "Por favor, selecione um Ano válido."
             print(f"! Erro de Validação: {errmsg}")
             plt.figure(figsize=(10, 2))
             plt.text(0.5, 0.5, errmsg, ha='center', va='center', fontsize=12, color='orange')
             plt.axis('off')
             plt.show()
             return # Parar

        # --- Chamar a Função de Plotagem ---
        criar_grafico_top5_municipios(
            dados=dados_ano,
            ano=selected_ano,
            metrica=selected_metrica
        )
        print("<<< Atualização de gráfico concluída.")


# --- Manipulador para Clique no Botão Comparar Municípios ---
def on_compare_clicked(button_info):
    """Manipula cliques no botão 'Comparar Top 5 Produtos."""
    # Usar o gerenciador de contexto 'output_area
    with output_area:
        clear_output(wait=True) # Limpar saída anterior
        print(">>> Evento: Botão 'Comparar Top 5 Produtos' clicado. Gerando gráfico de comparação...") # Feedback para usuário

        # Obter valores atuais dos filtros comuns e dropdowns de comparação
        selected_ano = ano_dropdown.value
        selected_metrica = metrica_dropdown.value
        selected_mun1 = municipio1_dropdown.value
        selected_mun2 = municipio2_dropdown.value

        # --- Chamar a Função de Plotagem de Comparação ---
        # A lógica de validação é tratada dentro de criar_grafico_comparativo
        criar_grafico_comparativo(
            dados=dados_ano,
            ano=selected_ano,
            metrica=selected_metrica,
            municipio1_nome=selected_mun1,
            municipio2_nome=selected_mun2
        )
        print("<<< Atualização de gráfico concluída.")


print("Célula 9/11: Funções de manipulador de eventos definidas.")

In [ ]:
# ===============================
# CÉLULA 10: Observar Mudanças e Cliques nos Widgets
# ===============================
# Conectar os widgets definidos na Célula 6 às suas respectivas funções de manipulador definidas na Célula 9.

print("Anexando manipuladores de eventos aos widgets...")

# --- Observar Mudanças nos Valores dos Dropdowns ---
# Quando o 'value' desses dropdowns mudar, chamar a função 'on_dropdown_change'.
# Este manipulador atualiza o gráfico de *único* município.
try:
    ano_dropdown.observe(on_dropdown_change, names='value')
    municipio_dropdown.observe(on_dropdown_change, names='value') # Para a visualização única
    metrica_dropdown.observe(on_dropdown_change, names='value')
    print("Observadores anexados a ano_dropdown, municipio_dropdown, metrica_dropdown.")
except NameError as e:
    print(f"Aviso: Não foi possível anexar observadores. Widget provavelmente não definido. Erro: {e}")


# --- Atribuir Manipuladores de Clique aos Botões ---
# Quando esses botões forem clicados, chamar suas respectivas funções de manipulador.
try:
    top5_municipios_button.on_click(on_top5_municipios_clicked)
    print("Manipulador de clique anexado a top5_municipios_button.")
except NameError as e:
     print(f"Aviso: Não foi possível anexar manipulador de clique a top5_municipios_button. Erro: {e}")

try:
    compare_button.on_click(on_compare_clicked)
    print("Manipulador de clique anexado a compare_button.")
except NameError as e:
     print(f"Aviso: Não foi possível anexar manipulador de clique a compare_button. Erro: {e}")


print("\nCélula 10/11: Observadores e manipuladores de clique anexados aos widgets.")
print("--> Próxima célula exibirá o painel.")

In [ ]:
# ===============================
# CÉLULA 11: Layout e Exibição Inicial (Corrigida)
# ===============================
# Organizar os widgets interativos usando VBox (vertical) e HBox (horizontal) como contêineres de layout.
# Exibir os widgets organizados e a área de saída.
# Finalmente, acionar a exibição do gráfico inicial com base nos valores padrão dos widgets.

print("Organizando layout do painel...")

# --- Definir Contêineres de Layout ---

# Contêiner para filtros comuns (Ano, Métrica)
common_filters = widgets.VBox([
    widgets.HTML("<b>Filtros Comuns:</b>", layout=widgets.Layout(margin='0 0 5px 0')), # Adicionar botão de margem
    ano_dropdown,
    metrica_dropdown
], layout=widgets.Layout(margin='0 10px 0 0')) # Adicionar margem direita a toda a caixa

# Contêiner para controles de visualização de município único
controls_single = widgets.VBox([
    widgets.HTML("<b>Visualizar por Município:</b>", layout=widgets.Layout(margin='0 0 5px 0')),
    municipio_dropdown
    # Nota: O gráfico é atualizado automaticamente quando municipio_dropdown muda (via observer)
], layout=widgets.Layout(margin='0 10px 0 0')) # Adicionar margem direita

# Contêiner para o botão "Top 5 Municípios SP"
controls_top_mun = widgets.VBox([
     widgets.HTML("<b>Visualizar Top Municípios (SP):</b>", layout=widgets.Layout(margin='15px 0 5px 0')), # Adicionar margem superior para espaçamento
     top5_municipios_button
     # Usa Ano e Métrica selecionados em common_filters
])

# Contêiner para controles de comparação
controls_compare = widgets.VBox([
    widgets.HTML("<b>Comparar Municípios:</b>", layout=widgets.Layout(margin='15px 0 5px 0')), # Adicionar margem superior
    municipio1_dropdown,
    municipio2_dropdown,
    compare_button
    # Usa Ano e Métrica selecionados em common_filters
])

# --- Organizar os Contêineres ---
# Exemplo 2: Agrupar controles logicamente em duas colunas
left_column = widgets.VBox([common_filters, controls_single])
right_column = widgets.VBox([controls_top_mun, controls_compare])
dashboard_controls = widgets.HBox([left_column, right_column])


print("Layout organizado.")

# --- Exibir o Painel ---
print("\n--- Exibindo Painel ---")
display(dashboard_controls)
print("--- Área de Saída (Gráficos aparecerão abaixo) ---")
display(output_area) # Exibir a área de saída onde os gráficos serão renderizados

# --- Acionar o Gráfico Inicial ---
# Isso é executado *após* os widgets serem exibidos.
# Usar o gerenciador de contexto 'with output_area' para garantir que o gráfico inicial apareça dentro dele.
with output_area:
    # clear_output(wait=True) # Garantir que está limpo antes do primeiro gráfico
    print("--- Gerando Gráfico Inicial (baseado nos padrões) ---")

    # Obter valores padrão dos widgets para gerar a primeira visualização
    initial_ano = ano_dropdown.value
    initial_municipio = municipio_dropdown.value
    initial_metrica = metrica_dropdown.value

    # Verificar se os valores padrão são válidos antes de tentar plotar
    if initial_ano and initial_municipio and \
       initial_municipio not in ['-- N/A --', 'Select Municipality']:
        # Se os padrões forem válidos, chamar a função de gráfico de município único
        criar_grafico_municipio_unico(
            dados=dados_ano,
            municipio_nome=initial_municipio,
            ano=initial_ano,
            metrica=initial_metrica
        )
    else:
        # Se os padrões não forem válidos (ex: falha no carregamento de dados), exibir mensagem
        warning_msg = "Não foi possível gerar o gráfico inicial.\n" \
                      "Verifique se os dados foram carregados corretamente (Células 3-5)\n" \
                      "e se há opções válidas selecionadas nos menus."
        # FIX: Realizar substituição antes do f-string
        log_msg = warning_msg.replace('\n', ' ')
        print(f"  ! Info: {log_msg}")
        # Exibir a mensagem dentro da área de saída usando um gráfico simples
        plt.figure(figsize=(10, 3))
        plt.text(0.5, 0.5, warning_msg, ha='center', va='center', fontsize=12, color='orange')
        plt.axis('off')
        plt.show()

print("\nCélula 11/11: Painel exibido. Gráfico inicial gerado (se possível).")
print("==========================================")
print(">>> Painel está pronto para uso! <<<")
print("==========================================")

In [ ]:
def clean_metadata():
    display(Javascript('''
    try {
      const md = JSON.parse(JSON.stringify(IPython.notebook.metadata));
      if (md.widgets) {
        console.log('Limpando metadados de widgets...');
        delete md.widgets;
      }
      IPython.notebook.metadata = md;
      IPython.notebook.save();
    } catch(err) {
      console.log('Nenhum widget persistente encontrado');
    }
    '''))

# Execute esta função antes de exportar
clean_metadata()